In [25]:
import cv2
import os

import numpy as np

app_dir = '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor'

In [75]:
def convert_map_to_road_map(ego_map):
    mask = (ego_map[0,:,:] == 1) * (ego_map[1,:,:] == 1) * (ego_map[2,:,:] == 1)
    # mask = (ego_map[:,:, 0] == 255) * (ego_map[:,:, 1] == 255) * (ego_map[:,:, 2] == 255)

    return (~mask)

import scipy.misc


def generate_scene_video(scene):
    all_array = []

    samples = list(range(126))
    image_dir = os.path.join(app_dir, 'artifacts/data/camera_data')
    video_dir = os.path.join(app_dir, 'artifacts/videos')
    size = ()

    for sample in samples:
        sample_dir = os.path.join(image_dir, 'scene_' + str(scene), 'sample_' + str(sample))
        image_front = cv2.imread(os.path.join(sample_dir, 'CAM_FRONT.jpeg'))
        image_front_left = cv2.imread(os.path.join(sample_dir, 'CAM_FRONT_LEFT.jpeg'))
        image_front_right = cv2.imread(os.path.join(sample_dir, 'CAM_FRONT_RIGHT.jpeg'))
        image_back = cv2.imread(os.path.join(sample_dir, 'CAM_BACK.jpeg'))
        image_back_left = cv2.imread(os.path.join(sample_dir, 'CAM_BACK_LEFT.jpeg'))
        image_back_right = cv2.imread(os.path.join(sample_dir, 'CAM_BACK_RIGHT.jpeg'))
        image_ego = cv2.imread(os.path.join(sample_dir, 'ego.png'))
        
        # image_ego = convert_map_to_road_map(image_ego)

        h, w, c = image_front.shape
        size = (int(3 * w / 2), int(2 * h / 2))
        full_image_size = (int(3 * w), int(2 * h / 2))

        front_all = np.concatenate((image_front_left, image_front, image_front_right), axis = 1)
        back_all = np.concatenate((image_back_left, image_back, image_back_right), axis = 1)

        image_all = np.concatenate((front_all, back_all), axis = 0)
        image_all = cv2.resize(image_all, size)
        image_ego = cv2.resize(image_ego, size)

        image_full = np.concatenate((image_all, image_ego), axis = 1)
        all_array.append(image_full)

    video_path = os.path.join(video_dir, f'scene_{scene}.avi')
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'DIVX'), 15, full_image_size)

    for i in range(len(all_array)):
        out.write(all_array[i])
    out.release()

In [76]:
generate_scene_video(123)
